# Basic experiments

Experiments comparing Random Isolation Similarity Forest to other outlier (anomaly) detection algorithms

In [1]:
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '..')
from data.data_getter import get_numerical_datasets

from sklearn.metrics import roc_auc_score

from tqdm import tqdm

In [2]:
# from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score

from notebooks.utils import *

We will use different outlier detection algorithms to compare to RISF:
* LOF
* ECOD
* Isolation Forest
* HBOS


In [3]:
SEED = 23

We will measure AUC (as a binary classification task of being an outlier) and processing time. We can show plots for every algorithm and the top-N feature importance

In [4]:
clfs_names = ['ECOD', 'LOF', 'IForest', 'HBOS', 'RISF']
results = {x: {} for x in clfs_names}

In [5]:
timer = Timer(timer_type="long_running")

In [6]:
datasets_loop = tqdm(get_numerical_datasets(), desc="Datasets (outer loop)", position=0)
algorithms_loop = tqdm(clfs_names, desc=" Algorithms (inner loop)", position=1, leave=False)

Datasets (outer loop): 0it [00:00, ?it/s]

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
for data in datasets_loop:
    datasets_loop.set_description(data['name'])
    for clf_name in algorithms_loop:
        algorithms_loop.set_description(clf_name)
        clf = new_clf(clf_name, SEED)
        timer.start()
        clf.fit(data['X_train'])
        timer.stop()
        train_time = timer.time_sec
        
        # get the prediction labels and outlier scores of the training and tests  data
        if clf_name == 'RISF': # other libs return sklearn UndefinedMetricWarning from predicting th train data
            y_train_pred = clf.predict(data['X_train']) # binary labels (0: inliers, 1: outliers)
        else:
            y_train_pred = clf.labels_
        
        timer.start()
        y_test_pred = clf.predict(data['X_test'])
        timer.stop()
        test_time = timer.time_sec

        if np.isnan(y_train_pred).any():
            results[clf_name][data['name']] = (np.nan, np.nan, np.nan, np.nan, 
                                               np.nan, np.nan, np.nan, np.nan) 
                                               # AUC/ROC, Rank@N for train,test ; fit Time
            continue
        
        
        
        roc_train=np.round(roc_auc_score(data['y_train'], y_train_pred), decimals=4)
        precision_train=np.round(precision_score(data['y_train'], y_train_pred), decimals=4)
        recall_train=np.round(recall_score(data['y_train'], y_train_pred), decimals=4)
        # prn_train=np.round(precision_n_scores(data['y_train'], y_train_pred), decimals=4) # will return to it when scores will be available quickly
        roc_test=np.round(roc_auc_score(data['y_test'], y_test_pred), decimals=4)
        precision_train=np.round(precision_score(data['y_train'], y_train_pred), decimals=4)
        recall_train=np.round(recall_score(data['y_train'], y_train_pred), decimals=4)
        # prn_test=np.round(precision_n_scores(data['y_test'], y_test_pred), decimals=4) # will return to it when scores will be available quickly

        results[clf_name][data['name']] = (roc_train, precision_train, recall_train,
                                           roc_test, precision_train, recall_train,
                                           train_time, train_time)


47_yeast.npz: : 47it [3:31:20, 269.79s/it]       


In [9]:
df = pd.DataFrame(results)
df.to_pickle('../results/numerical_final.pkl')
df

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,"(0.4992, 0.03, 0.0985, 0.5021, 0.03, 0.0985, 0...","(0.6169, 0.0995, 0.3267, 0.6005, 0.0995, 0.326...","(0.5119, 0.0375, 0.1231, 0.5092, 0.0375, 0.123...","(0.5119, 0.0375, 0.1231, 0.5153, 0.0375, 0.123...","(0.502, 0.0411, 0.0152, 0.5014, 0.0411, 0.0152..."
02_annthyroid.npz,"(0.6467, 0.2758, 0.3717, 0.6106, 0.2758, 0.371...","(0.6279, 0.25, 0.3369, 0.6441, 0.25, 0.3369, 0...","(0.6482, 0.2778, 0.3743, 0.6131, 0.2778, 0.374...","(0.6323, 0.256, 0.3449, 0.5929, 0.256, 0.3449,...","(0.6424, 0.3232, 0.3422, 0.6002, 0.3232, 0.342..."
03_backdoor.npz,"(0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 2.453, 2.453)","(0.6915, 0.1157, 0.4736, 0.6812, 0.1157, 0.473...","(0.5638, 0.0548, 0.2245, 0.5627, 0.0548, 0.224...","(0.5824, 0.0637, 0.2607, 0.5759, 0.0637, 0.260...","(0.4933, 0.0187, 0.0423, 0.4945, 0.0187, 0.042..."
04_breastw.npz,"(0.6407, 1.0, 0.2814, 0.6458, 1.0, 0.2814, 0.0...","(0.4504, 0.125, 0.0359, 0.4616, 0.125, 0.0359,...","(0.6391, 0.9792, 0.2814, 0.6111, 0.9792, 0.281...","(0.6299, 0.9375, 0.2695, 0.6944, 0.9375, 0.269...","(0.9554, 0.9349, 0.9461, 0.9112, 0.9349, 0.946..."
05_campaign.npz,"(0.6529, 0.4183, 0.3713, 0.6506, 0.4183, 0.371...","(0.4849, 0.0826, 0.0733, 0.4767, 0.0826, 0.073...","(0.6107, 0.334, 0.2965, 0.604, 0.334, 0.2965, ...","(0.6577, 0.428, 0.3799, 0.6451, 0.428, 0.3799,...","(0.6879, 0.2151, 0.7004, 0.6867, 0.2151, 0.700..."
06_cardio.npz,"(0.7325, 0.5, 0.5203, 0.7703, 0.5, 0.5203, 0.0...","(0.5706, 0.2188, 0.2276, 0.572, 0.2188, 0.2276...","(0.737, 0.5078, 0.5285, 0.7966, 0.5078, 0.5285...","(0.71, 0.4609, 0.4797, 0.7494, 0.4609, 0.4797,...","(0.6677, 0.4653, 0.3821, 0.747, 0.4653, 0.3821..."
07_Cardiotocography.npz,"(0.603, 0.5743, 0.2607, 0.6217, 0.5743, 0.2607...","(0.5185, 0.2838, 0.1288, 0.5223, 0.2838, 0.128...","(0.5932, 0.5405, 0.2454, 0.5824, 0.5405, 0.245...","(0.5735, 0.473, 0.2147, 0.5433, 0.473, 0.2147,...","(0.5898, 0.5887, 0.2239, 0.5513, 0.5887, 0.223..."
08_celeba.npz,"(0.6722, 0.098, 0.4367, 0.6672, 0.098, 0.4367,...","(0.4725, 0.0104, 0.0462, 0.4698, 0.0104, 0.046...","(0.6105, 0.0709, 0.3161, 0.6007, 0.0709, 0.316...","(0.6708, 0.0974, 0.4339, 0.6644, 0.0974, 0.433...","(0.4629, 0.0201, 0.6368, 0.4597, 0.0201, 0.636..."
09_census.npz,"(0.5048, 0.0677, 0.1091, 0.5048, 0.0677, 0.109...","(0.4724, 0.03, 0.0483, 0.4742, 0.03, 0.0483, 3...","(0.4889, 0.0491, 0.0791, 0.4888, 0.0491, 0.079...","(0.5012, 0.0634, 0.1022, 0.5001, 0.0634, 0.102...","(0.5519, 0.0808, 0.4184, 0.5439, 0.0808, 0.418..."
10_cover.npz,"(0.7882, 0.0644, 0.6708, 0.8052, 0.0644, 0.670...","(0.5561, 0.0203, 0.2111, 0.5602, 0.0203, 0.211...","(0.6175, 0.032, 0.3328, 0.6285, 0.032, 0.3328,...","(0.586, 0.026, 0.2704, 0.5933, 0.026, 0.2704, ...","(0.7667, 0.0477, 0.6615, 0.7858, 0.0477, 0.661..."


In [11]:
multiTable = df.copy()

for name in clfs_names:
    multiTable[[(name, 'TrainROC'), 
        (name, 'TrainPrecision'),
        (name, 'TrainRecall'), 
        (name, 'TestROC'), 
        (name, 'TestPrecision'),
        (name, 'TestRecall'), 
        (name, 'TrainTime [s]'),
        (name, 'PredictTime [s]')]
        ] = pd.DataFrame(multiTable[name].to_list(), index=multiTable.index)

multiTable= multiTable.drop(clfs_names, axis=1)
multiTable.columns = pd.MultiIndex.from_tuples(multiTable.columns)
multiTable = multiTable.style.set_caption('All performed experiments').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
multiTable

In [19]:
train_times = df.apply(lambda x: x.str[6])
train_times = train_times.style.highlight_min(color = 'green', axis = 1).set_caption('Training time [s]').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
train_times   

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.344000,3.422000,3.031000,5.031000,2.813000
02_annthyroid.npz,0.015000,0.109000,0.343000,0.015000,3.157000
03_backdoor.npz,2.453000,14.266000,27.625000,0.844000,10.594000
04_breastw.npz,0.016000,0.000000,0.171000,0.828000,1.297000
05_campaign.npz,0.313000,1.359000,4.438000,0.094000,1.610000
06_cardio.npz,0.000000,0.141000,0.297000,0.000000,4.797000
07_Cardiotocography.npz,0.015000,0.047000,0.281000,0.000000,5.093000
08_celeba.npz,1.141000,327.750000,17.828000,1.203000,1.047000
09_census.npz,24.813000,331.703000,281.593000,8.547000,6.281000
10_cover.npz,1.032000,9.297000,11.844000,4.844000,7.609000


In [20]:
test_times = df.apply(lambda x: x.str[7])
test_times = test_times.style.highlight_min(color = 'green', axis = 1).set_caption('Prediction time [s]').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
test_times   

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.344000,3.422000,3.031000,5.031000,2.813000
02_annthyroid.npz,0.015000,0.109000,0.343000,0.015000,3.157000
03_backdoor.npz,2.453000,14.266000,27.625000,0.844000,10.594000
04_breastw.npz,0.016000,0.000000,0.171000,0.828000,1.297000
05_campaign.npz,0.313000,1.359000,4.438000,0.094000,1.610000
06_cardio.npz,0.000000,0.141000,0.297000,0.000000,4.797000
07_Cardiotocography.npz,0.015000,0.047000,0.281000,0.000000,5.093000
08_celeba.npz,1.141000,327.750000,17.828000,1.203000,1.047000
09_census.npz,24.813000,331.703000,281.593000,8.547000,6.281000
10_cover.npz,1.032000,9.297000,11.844000,4.844000,7.609000


In [14]:
train_roc = df.apply(lambda x: x.str[0])
train_roc = train_roc.style.highlight_max(color = 'green', axis = 1).set_caption('AUC/ROC training set').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
train_roc

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.499200,0.616900,0.511900,0.511900,0.502000
02_annthyroid.npz,0.646700,0.627900,0.648200,0.632300,0.642400
03_backdoor.npz,0.500000,0.691500,0.563800,0.582400,0.493300
04_breastw.npz,0.640700,0.450400,0.639100,0.629900,0.955400
05_campaign.npz,0.652900,0.484900,0.610700,0.657700,0.687900
06_cardio.npz,0.732500,0.570600,0.737000,0.710000,0.667700
07_Cardiotocography.npz,0.603000,0.518500,0.593200,0.573500,0.589800
08_celeba.npz,0.672200,0.472500,0.610500,0.670800,0.462900
09_census.npz,0.504800,0.472400,0.488900,0.501200,0.551900
10_cover.npz,0.788200,0.556100,0.617500,0.586000,0.766700


In [21]:
train_prec = df.apply(lambda x: x.str[1])
train_prec = train_prec.style.highlight_max(color = 'green', axis = 1).set_caption('Training set precision').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
train_prec

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.030000,0.099500,0.037500,0.037500,0.041100
02_annthyroid.npz,0.275800,0.250000,0.277800,0.256000,0.323200
03_backdoor.npz,0.000000,0.115700,0.054800,0.063700,0.018700
04_breastw.npz,1.000000,0.125000,0.979200,0.937500,0.934900
05_campaign.npz,0.418300,0.082600,0.334000,0.428000,0.215100
06_cardio.npz,0.500000,0.218800,0.507800,0.460900,0.465300
07_Cardiotocography.npz,0.574300,0.283800,0.540500,0.473000,0.588700
08_celeba.npz,0.098000,0.010400,0.070900,0.097400,0.020100
09_census.npz,0.067700,0.030000,0.049100,0.063400,0.080800
10_cover.npz,0.064400,0.020300,0.032000,0.026000,0.047700


In [23]:
train_rec = df.apply(lambda x: x.str[2])
train_rec = train_rec.style.highlight_max(color = 'green', axis = 1).set_caption('Training set recall').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
train_rec

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.098500,0.326700,0.123100,0.123100,0.015200
02_annthyroid.npz,0.371700,0.336900,0.374300,0.344900,0.342200
03_backdoor.npz,0.000000,0.473600,0.224500,0.260700,0.042300
04_breastw.npz,0.281400,0.035900,0.281400,0.269500,0.946100
05_campaign.npz,0.371300,0.073300,0.296500,0.379900,0.700400
06_cardio.npz,0.520300,0.227600,0.528500,0.479700,0.382100
07_Cardiotocography.npz,0.260700,0.128800,0.245400,0.214700,0.223900
08_celeba.npz,0.436700,0.046200,0.316100,0.433900,0.636800
09_census.npz,0.109100,0.048300,0.079100,0.102200,0.418400
10_cover.npz,0.670800,0.211100,0.332800,0.270400,0.661500


In [25]:
test_roc = df.apply(lambda x: x.str[3])
test_roc = test_roc.style.highlight_max(color = 'green', axis = 1).set_caption('AUC/ROC on test set').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
test_roc

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.502100,0.600500,0.509200,0.515300,0.501400
02_annthyroid.npz,0.610600,0.644100,0.613100,0.592900,0.600200
03_backdoor.npz,0.500000,0.681200,0.562700,0.575900,0.494500
04_breastw.npz,0.645800,0.461600,0.611100,0.694400,0.911200
05_campaign.npz,0.650600,0.476700,0.604000,0.645100,0.686700
06_cardio.npz,0.770300,0.572000,0.796600,0.749400,0.747000
07_Cardiotocography.npz,0.621700,0.522300,0.582400,0.543300,0.551300
08_celeba.npz,0.667200,0.469800,0.600700,0.664400,0.459700
09_census.npz,0.504800,0.474200,0.488800,0.500100,0.543900
10_cover.npz,0.805200,0.560200,0.628500,0.593300,0.785800


In [27]:
test_prec = df.apply(lambda x: x.str[4])
test_prec = test_prec.style.highlight_max(color = 'green', axis = 1).set_caption('Test set precision').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
test_prec

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.030000,0.099500,0.037500,0.037500,0.041100
02_annthyroid.npz,0.275800,0.250000,0.277800,0.256000,0.323200
03_backdoor.npz,0.000000,0.115700,0.054800,0.063700,0.018700
04_breastw.npz,1.000000,0.125000,0.979200,0.937500,0.934900
05_campaign.npz,0.418300,0.082600,0.334000,0.428000,0.215100
06_cardio.npz,0.500000,0.218800,0.507800,0.460900,0.465300
07_Cardiotocography.npz,0.574300,0.283800,0.540500,0.473000,0.588700
08_celeba.npz,0.098000,0.010400,0.070900,0.097400,0.020100
09_census.npz,0.067700,0.030000,0.049100,0.063400,0.080800
10_cover.npz,0.064400,0.020300,0.032000,0.026000,0.047700


In [28]:
test_rec = df.apply(lambda x: x.str[5])
test_rec = test_rec.style.highlight_max(color = 'green', axis = 1).set_caption('Test set recall').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '16px'),
        ('font-weight', 'bold')
    ]
}])
test_rec

,ECOD,LOF,IForest,HBOS,RISF
01_ALOI.npz,0.098500,0.326700,0.123100,0.123100,0.015200
02_annthyroid.npz,0.371700,0.336900,0.374300,0.344900,0.342200
03_backdoor.npz,0.000000,0.473600,0.224500,0.260700,0.042300
04_breastw.npz,0.281400,0.035900,0.281400,0.269500,0.946100
05_campaign.npz,0.371300,0.073300,0.296500,0.379900,0.700400
06_cardio.npz,0.520300,0.227600,0.528500,0.479700,0.382100
07_Cardiotocography.npz,0.260700,0.128800,0.245400,0.214700,0.223900
08_celeba.npz,0.436700,0.046200,0.316100,0.433900,0.636800
09_census.npz,0.109100,0.048300,0.079100,0.102200,0.418400
10_cover.npz,0.670800,0.211100,0.332800,0.270400,0.661500


In [30]:
import dataframe_image as dfi

In [32]:
dfi.export(train_roc, '../results/figures/train_roc.png')
dfi.export(train_prec, '../results/figures/train_precision.png')
dfi.export(train_rec, '../results/figures/train_recall.png')
dfi.export(test_roc, '../results/figures/test_roc.png')
dfi.export(test_prec, '../results/figures/test_precision.png')
dfi.export(test_rec, '../results/figures/test_recall.png')
dfi.export(train_times, '../results/figures/train_times.png')
dfi.export(test_times, '../results/figures/train_times.png')
# dfi.export(multiTable, '../results/figures/multiTable.png')